In [785]:
import pandas as pd 
import numpy as np
import tensorly as tl
import plotly.express as px
import matplotlib.pyplot as plt 
from tensorly.decomposition import CP,tucker, parafac, non_negative_tucker
from datetime import timedelta, date

In [1066]:
df_FR = pd.read_excel('FINANCIAL_RATIOS_NEW.xlsx', index_col =[0], skiprows=[1])

Index(['AMAT US Equity', 'COKE US Equity', 'WDFC US Equity', 'AAPL US Equity',
       'KLAC US Equity', 'SEIC US Equity', 'CSPI US Equity', 'ALOT US Equity',
       'AMGN US Equity', 'CAMP US Equity',
       ...
       'DVN US Equity', 'ORCL US Equity', 'PG US Equity', 'INTC US Equity',
       'TTMI US Equity', 'TTEK US Equity', 'WFC US Equity', 'WMT US Equity',
       'C US Equity', 'BA US Equity'],
      dtype='object', length=220)

## Storing values from dataframe into a dictionary

In [1069]:
# df_FR = pd.read_excel('FINANCIAL_RATIOS_f.xlsx', index_col =[0])
price = None
FR_dic = {}
FR_list = ['PE', 'PX_LAST', 'PS','PB']
n = 0 
NUMBER_OF_STOCKS = 220
for i,name in enumerate(FR_list) : 
    FR_dic[name] = (df_FR.iloc[:,n:n+NUMBER_OF_STOCKS]
                    .set_index(df_FR.index)
                    )

    # if name not in ['PX_LAST', 'MCAP'] : 
        # FR_dic[name] = (FR_dic[name] - FR_dic[name].mean()) / FR_dic[name].std()
        
    if name in ['PX_LAST'] :

        price = FR_dic['PX_LAST']
        FR_dic[name] = np.log(FR_dic[name])
        FR_dic[name] = FR_dic[name].diff()
    

    
    FR_dic[name] = FR_dic[name][1:]
    n += NUMBER_OF_STOCKS + 2

COLUMN_NAMES = FR_dic['PE'].columns
FR_index = FR_dic['PE'].index


In [1070]:
df_TI = pd.read_excel('TECH_NEW.xlsx', skiprows=[0])

TI_dic = {}
TI_list = ['BB_PERCENT', 'HURST', 'MOM', 'ROC','RSI','WLPR']
n = 1

for _, TI in enumerate(TI_list) : 
    filter_col = [col for col in df_TI if col.startswith(TI)]
    TI_dic[TI] = (df_TI[filter_col]
                 .set_axis(COLUMN_NAMES, axis=1)
                 .set_index(df_TI['Dates'])
                 .iloc[:-1]
                 .fillna(method = 'ffill')
                 .set_index(FR_index)
                 )
    # TI_dic[TI] = (TI_dic[TI] - TI_dic[TI].mean()) / TI_dic[TI].std()
TI_dic.pop('ROC', None)
TI_dic.pop('WLPR', None)
TI_dic.pop('BB_PERCENT', None)



,AMAT US Equity,COKE US Equity,WDFC US Equity,AAPL US Equity,KLAC US Equity,SEIC US Equity,CSPI US Equity,ALOT US Equity,AMGN US Equity,CAMP US Equity,...,DVN US Equity,ORCL US Equity,PG US Equity,INTC US Equity,TTMI US Equity,TTEK US Equity,WFC US Equity,WMT US Equity,C US Equity,BA US Equity
2006-11-17,0.920725,0.790533,0.299094,0.889780,0.953225,0.829484,0.663764,0.029398,0.753526,0.656345,...,0.978271,0.866047,0.734660,0.968789,0.588856,0.972808,0.671097,0.610858,0.752871,1.042221
2006-11-24,0.823815,0.767193,0.260161,0.995344,1.022082,0.831057,0.618387,0.216084,0.663835,0.700463,...,0.924439,0.896947,0.767123,0.828099,0.702060,0.796085,0.430650,0.642886,0.792155,1.107741
2006-12-01,0.745419,0.718539,0.309934,0.991250,0.876556,0.762449,0.622976,0.273581,0.624597,0.792960,...,0.984328,0.796322,0.703897,0.723577,0.593705,0.607564,0.373045,0.508127,0.576063,0.960095
2006-12-08,0.806834,0.853188,0.324167,0.914653,0.890031,0.792233,0.708426,0.522080,0.380172,0.885711,...,0.954400,0.589684,0.761502,0.647949,0.641447,0.645454,0.393740,0.454518,0.769086,0.989226
2006-12-15,0.648235,0.911753,0.327228,0.854121,0.728860,0.751649,0.470256,0.407438,0.374138,0.839280,...,0.787630,0.592830,0.706313,0.593542,0.302576,0.699298,0.330078,0.360238,1.079196,0.877894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-29,0.013064,0.165564,0.232073,0.023449,0.066688,0.154276,0.069477,0.159267,0.855918,0.103740,...,0.661348,0.198829,0.579860,0.117454,0.388168,0.159488,0.028132,0.843362,0.069082,-0.152900
2022-05-06,0.216839,0.338704,0.215706,0.379180,0.332483,0.295268,0.372435,0.259133,0.548082,0.227268,...,0.972695,0.173900,0.471805,0.256990,0.650765,0.195110,0.145202,0.826072,0.211097,-0.014661
2022-05-13,0.029507,0.366630,0.173864,-0.151802,0.108861,-0.126248,0.170237,0.141725,0.638176,-0.035569,...,0.835199,0.002971,0.231608,0.022231,0.490767,-0.117161,-0.001853,0.588303,0.080008,-0.168925
2022-05-20,0.174819,0.341748,0.192042,-0.150375,0.282327,0.110766,0.529911,0.148865,0.699472,0.336541,...,0.987541,-0.053657,-0.069306,0.054102,0.657500,-0.018672,0.033662,-0.087700,0.179703,-0.073271


In [1002]:
look_back_duration = 2
look_forward = 2

### We attempt to predict 'test_company's' cumulative log returns 2 weeks from now based on historical data which has a lookback period of 13 weeks.


In [1004]:
# # import sys
# # import numpy as np 
# # np.set_printoptions(threshold=sys.maxsize)

# ft1 =  np.stack(FR_dic.values(), axis=2)
# ft2 = np.stack(TI_dic.values(), axis=2)
# feature_tensor = np.dstack((ft1, ft2))

# error_list = []
# for n in np.arange(1,150,2) : 
#     core, factors = tucker(feature_tensor, rank= [711,n,6])
#     pc = tl.tenalg.mode_dot(feature_tensor, factors[1].T, mode = 1)
#     rec = tl.tucker_to_tensor((core,factors))
#     rec_error = tl.norm(rec - feature_tensor)/tl.norm(feature_tensor)
#     error_list.append(rec_error)
# fig, ax = plt.subplots(1)
# ax.plot(np.arange(1,150,2), error_list)
# ax.scatter(np.arange(1,150,2), error_list)
# ax.grid('On')
# ax.set_title("Reconstruction error against number of companies")
# ax.set_xlabel("Rank")
# ax.set_ylabel("Reconstruction error")



From the figure above, the reconstruction error decreases drastically initially once the rank increases above 13, the error no longer drastically anymore. We will hence select a rank of [700,18,6]

In [1017]:
import sys
import numpy as np 
np.set_printoptions(threshold=sys.maxsize)

# FR_dic.pop('PX_LAST',None)


ft1 =  np.stack(FR_dic.values(), axis=2)
ft2 = np.stack(TI_dic.values(), axis=2)
feature_tensor = np.dstack((ft1, ft2))
feature_tensor, feature_tensor2 = feature_tensor[:-200], feature_tensor[-200:]
n_stocks = 20


def tuck_decomp(feature_tensor, t , s , f) : 

    core, factors = tucker(feature_tensor, rank= [t,s,f]) # 811 time, 20 stocks, 7 
    pc = tl.tenalg.mode_dot(feature_tensor, factors[1].T, mode = 1)

    rec = tl.tucker_to_tensor((core,factors))
    rec_error = tl.norm(rec - feature_tensor)/tl.norm(feature_tensor)
    print(rec_error)
    return pc

def first_order_diff(pc, length, mode = 2) : 
    # for n in range(length) : 
    #     plt.plot(pc[:,n,2])
    for i in range(len(pc)-1) :
            tmp = pc[i+1,:,mode] - pc[i,:,mode]
            pc[i,:,mode] = tmp
    pc[1:,:,mode] = pc[:-1,:,mode]
    
    return pc 

pc = tuck_decomp(feature_tensor,len(feature_tensor), 20, 7)
pc = first_order_diff(pc,n_stocks,mode=0)
pc = first_order_diff(pc,n_stocks,mode=1)
pc = first_order_diff(pc,n_stocks,mode=2)
pc = first_order_diff(pc,n_stocks,mode=3)
pc = first_order_diff(pc,n_stocks,mode=4)
pc = first_order_diff(pc,n_stocks,mode=5)
pc = first_order_diff(pc,n_stocks,mode=6)

pc2 = tuck_decomp(feature_tensor2, len(feature_tensor2), 20, 7)
pc2 = first_order_diff(pc2,n_stocks,mode=0)
pc2 = first_order_diff(pc2,n_stocks,mode=1)
pc2 = first_order_diff(pc2,n_stocks,mode=2)
pc2 = first_order_diff(pc2,n_stocks,mode=3)
pc2 = first_order_diff(pc2,n_stocks,mode=4)
pc2 = first_order_diff(pc2,n_stocks,mode=5)
pc2 = first_order_diff(pc2,n_stocks,mode=6)

pc = pc[1:, : ,:]
pc2 = pc2[1:,:,:]

diff = True



# for i in range(7) : 
#     plt.figure()
#     for n in range(n_stocks) :
#         plt.plot(pc[:,n,i])



list_of_features = [] #Split the features into different look_back_duration time slots. 
for i in range(0,len(pc)-look_back_duration-look_forward+1) : 
    list_of_features.append(pc[i:i+look_back_duration, :,:])


list_of_features2 = [] #Split the features into different look_back_duration time slots. 
for i in range(0,len(pc2)-look_back_duration-look_forward+1) : 
    list_of_features2.append(pc2[i:i+look_back_duration, :,:])


print(len(list_of_features))
print(len(list_of_features2))

def get_solo_features(solo_df, lb_duration, lf_duration) :
    list_of_solo_f = []
    #SOLO FEATURES FOR AAPL
    for i in range(0,len(solo_df)-lb_duration-lf_duration+1) : 
        list_of_solo_f.append(solo_df[i:i+lb_duration].to_numpy())
    return list_of_solo_f

c:\Users\zackx\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


0.0726374165280644
0.08462169363092477
607
196


In [1055]:
#Append the training data of AAPL 
solo_df = pd.DataFrame()

test_company = 'HAL'
k = 0
for key,value in FR_dic.items() : 
    if k == 0 : 
        solo_df[key] = value[f'{test_company} US Equity']
    if k > 0 : 
        solo_df[key] = value[f'{test_company} US Equity'+ f'.{k}']
    k+=1

for key,value in TI_dic.items() : 
    solo_df[key] = value[f'{test_company} US Equity']


for col in FR_dic['PX_LAST'] : 
    if col.startswith(test_company) : 
        y_predict = FR_dic['PX_LAST'][col]

test_comp_returns = pd.DataFrame(y_predict).set_axis(['Log Returns'], axis = 1)
test_comp_returns['Cumulative Log Returns'] = test_comp_returns['Log Returns'].cumsum()
def get_n_week_retuns(log_returns, look_back_duration, lookforward) : 
    sdate = test_comp_returns.index.values[0]
    edate = test_comp_returns.index.values[-1]
    s = (log_returns
     .reset_index()
     .iloc[look_back_duration:]
    )
    n_week_retuns = s.rolling(lookforward).sum()
    n_week_retuns.index = list(pd.date_range(sdate ,edate + pd.to_timedelta(2, unit='D') ,freq='w') - pd.to_timedelta(2, unit='D'))[:-look_back_duration]
    return n_week_retuns.dropna() 


def to_simple_return(cum_log_ret) : 
    return np.exp(cum_log_ret) - 1




## Generate our target labels

1) Predict whether the stock will go +1 or -1 in the next 2 weeks.



In [1071]:
# x_ret_list contains the 
y_ret =  get_n_week_retuns(test_comp_returns['Log Returns'], look_back_duration = look_back_duration, lookforward = look_forward)

y_ret[(y_ret > 0)] = 1

y_ret[y_ret <= 0] = -1

y_ret1, y_ret2 = y_ret[:-200], y_ret[-200+look_back_duration+1:]


list_of_solo_f, list_of_solo_f2 = None, None 

if diff == True : 
    y_ret1 = y_ret1[1:]
    y_ret2 = y_ret2[1:]
    list_of_solo_f = get_solo_features(solo_df, look_back_duration, look_forward)
    list_of_solo_f, list_of_solo_f2 = (list_of_solo_f[:-200])[1:], (list_of_solo_f[-200:])[1:]
    # print(len(y_ret1))
    # print(len(y_ret2))
    # print(len(list_of_solo_f))
    # print(len(list_of_solo_f2))

    # print(len(list_of_features))
    # print(len(list_of_features2))
print(len(list_of_features))
print(len(list_of_solo_f))

list_of_combined_features = []
list_of_combined_features2 = []
for feature_t,solo_feature in zip(list_of_features,list_of_solo_f)  :
    solo_feature = solo_feature[:,newaxis,:]
    tmp = np.concatenate((feature_t,solo_feature), axis=1)
    list_of_combined_features.append(tmp)

for feature_t,solo_feature in zip(list_of_features2,list_of_solo_f2)  :
    solo_feature = solo_feature[:,newaxis,:]
    tmp = np.concatenate((feature_t,solo_feature), axis=1)
    list_of_combined_features2.append(tmp)


X_Cols = [tensor.flatten() for tensor in list_of_features ]

S_Cols = [mat.flatten() for mat in list_of_solo_f]

C_Cols = [tensor.flatten() for tensor in list_of_combined_features]


X_tot = [tensor.flatten() for tensor in list_of_features2 ]

S_tot = [mat.flatten() for mat in list_of_solo_f2]

C_tot = [tensor.flatten() for tensor in list_of_combined_features2]

X_train ,Y_train = X_Cols, y_ret1
X_test ,Y_test =   X_tot[-200:-100], y_ret2[:-100]
X_val ,Y_val =   X_tot[-100:], y_ret2[-100:]


C_train, Y_train = C_Cols, y_ret1
C_test ,Y_test =   C_tot[-200:-100], y_ret2[:-100]
C_val ,Y_val =   C_tot[-100:], y_ret2[-100:]


# Y_test = y_ret[-200:-100] 
# Y_val = y_ret[-100:]




# X_val1, S_val1, C_val1, Y_val1 = X_Cols[-120:], S_Cols[-120:], C_Cols[-120:], y_ret[-120:]

# X_val2, S_val2, C_val2, Y_val2 = X_val1[-60:], S_val1[-60:], C_val1[-60:], Y_val1[-60:]

607
607


### We build a RandomForestRegressor model 

Random Forests is a supervised machine learning algorithim that uses multiple decision trees in aggregate to help make more stable and accurate predictions.


In [1057]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from numpy import newaxis

#combined_features




# X_train, X_test, Y_train, Y_test = np.concatenate([X_Cols[:-350],X_Cols[:-150], X_Cols[len(X_Cols)-350:len(X_Cols)-150]]),  



 

In [1058]:
rand_frst_C = RandomForestClassifier(n_estimators= 50, min_samples_split = 2, min_samples_leaf=2,  max_features='auto', max_depth = 12, bootstrap = True, oob_score=True)
# rand_frst_X = RandomForestClassifier(n_estimators= 30, min_samples_split = 2, min_samples_leaf=2,  max_features='auto', max_depth = 10, bootstrap = True)

In [1059]:
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, mean_squared_error, classification_report

def baseline(Y_test, prediction) : 
    print(len(prediction))
    all_ones = np.ones(len(Y_test))
    all_neg_ones = np.ones(len(Y_test)) * -1

    print(f"Actual number of ones : {np.count_nonzero(Y_test == 1)}")
    print(f"Predicted number of ones : {np.count_nonzero(prediction == 1)}")
    # print(f"Baseline if all were ones : {np.count_nonzero((all_ones==Y_test)) / len(Y_test)}")

    print(f"Actual number of -1 : {np.count_nonzero(Y_test == -1)}")
    print(f"Predicted number of -1 : {np.count_nonzero(prediction == -1)}")
    # print(f"Baseline if all were -1 : {np.count_nonzero((all_neg_ones==Y_test)) / len(Y_test)}")

In [1073]:
## HYPERPARAMETER TUNING
n_estimators = [1000, 2000]

max_features = ['auto']

max_depth = [int(x) for x in np.linspace(3,13,4)]

min_samples_split = [2]

min_samples_leaf = [1,2]

bootstrap = [False, True]
random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap' : bootstrap

}

rf = RandomForestClassifier()

# Using 5 fold cross validation
# Search across 100 different combintations and use all available cores

rf_random = RandomizedSearchCV(
    estimator = rf,
    param_distributions= random_grid,
    n_iter = 40, 
    verbose= 4,
    cv = 3,
    random_state=520,
    n_jobs = - 1,

                        )
rf_random.fit(C_train, Y_train)


c:\Users\zackx\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 32 is smaller than n_iter=40. Running 32 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 32 candidates, totalling 96 fits


c:\Users\zackx\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
c:\Users\zackx\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=40,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [False, True],
                                        'max_depth': [3, 6, 9, 13],
                                        'max_features': ['auto'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2],
                                        'n_estimators': [1000, 2000]},
                   random_state=520, verbose=4)

In [1074]:
print('Random grid: ', random_grid, '\n')

print('Best Parameters: ', rf_random.best_params_, ' \n')
print()
print(rf_random.best_score_)
print(rf_random.best_params_)
target_names = ['Red', 'Green']
prediction = rf_random.predict(C_train)
print(accuracy_score(Y_train, prediction))
# print(classification_report(Y_train, prediction, target_names=target_names))
print(confusion_matrix(Y_train, prediction))


prediction = rf_random.predict(C_test)
print(accuracy_score(Y_test, prediction))
# print(classification_report(Y_test, prediction, target_names=target_names))
print(confusion_matrix(Y_test, prediction))


prediction_val1= rf_random.predict(C_val)
print(accuracy_score(Y_val, prediction_val1))
print(confusion_matrix(Y_val, prediction_val1))
# print(classification_report(Y_val1, prediction, target_names=target_names))

# print(prediction)



# prediction_val2 = rf_random.predict(C_val2)
# print(accuracy_score(Y_val2, prediction_val2))
# print(confusion_matrix(Y_val2, prediction_val2))
# print(classification_report(Y_val2, prediction, target_names=target_names))



# prediction = rf_random.predict(X_train)
# print(accuracy_score(Y_train, prediction))
# # print(classification_report(Y_train, prediction))
# print(confusion_matrix(Y_train, prediction))


# prediction = rf_random.predict(X_test)
# print(accuracy_score(Y_test, prediction))
# # print(classification_report(Y_test, prediction, target_names=target_names))
# print(confusion_matrix(Y_test, prediction))


# prediction = rf_random.predict(X_val)
# print(accuracy_score(Y_val, prediction))
# print(confusion_matrix(Y_val, prediction))
# # print(classification_report(Y_val1, prediction, target_names=target_names))

# print(prediction)


# prediction = rf_random.predict(S_train)
# print(accuracy_score(Y_train, prediction))
# # print(classification_report(Y_train, prediction, target_names=target_names))
# print(confusion_matrix(Y_train, prediction))


# prediction = rf_random.predict(S_test)
# print(accuracy_score(Y_test, prediction))
# # print(classification_report(Y_test, prediction, target_names=target_names))
# print(confusion_matrix(Y_test, prediction))


# prediction = rf_random.predict(S_val1)
# print(accuracy_score(Y_val1, prediction))
# print(confusion_matrix(Y_val1, prediction))
# # print(classification_report(Y_val1, prediction, target_names=target_names))

# print(prediction)



# prediction = rf_random.predict(S_val2)
# print(accuracy_score(Y_val2, prediction))
# print(confusion_matrix(Y_val2, prediction))
# # print(classification_report(Y_val2, prediction, target_names=target_names))










Random grid:  {'n_estimators': [1000, 2000], 'max_features': ['auto'], 'max_depth': [3, 6, 9, 13], 'min_samples_split': [2], 'min_samples_leaf': [1, 2], 'bootstrap': [False, True]} 

Best Parameters:  {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 3, 'bootstrap': False}  


nan
{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 3, 'bootstrap': False}
0.7973640856672158
[[154 123]
 [  0 330]]
0.5104166666666666
[[ 8 45]
 [ 2 41]]
0.46
[[10 32]
 [22 36]]


In [1062]:
baseline(Y_val1, prediction_val1)


baseline(Y_val2, prediction_val2)

100
Actual number of ones : 66
Predicted number of ones : 65
Actual number of -1 : 54
Predicted number of -1 : 35
60
Actual number of ones : 37
Predicted number of ones : 60
Actual number of -1 : 23
Predicted number of -1 : 0


In [1063]:
rand_frst_C.fit(C_train,Y_train)


prediction = rand_frst_C.predict(C_train)
print(accuracy_score(Y_train, prediction))
print(confusion_matrix(Y_train, prediction))

prediction = rand_frst_C.predict(C_test)
print(accuracy_score(Y_test, prediction))
print(confusion_matrix(Y_test, prediction))

prediction = rand_frst_C.predict(C_val1)
print(accuracy_score(Y_val1, prediction))
print(confusion_matrix(Y_val1, prediction))

prediction = rand_frst_C.predict(C_val2)
print(accuracy_score(Y_val2, prediction))
print(confusion_matrix(Y_val2, prediction))

baseline(Y_val1)

baseline(Y_val2)







C:\Users\zackx\AppData\Local\Temp/ipykernel_9904/1114118841.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rand_frst_C.fit(C_train,Y_train)


1.0
[[277   0]
 [  0 330]]
0.4375
[[18 35]
 [19 24]]
0.5166666666666667
[[16 38]
 [20 46]]
0.48333333333333334
[[ 5 18]
 [13 24]]


TypeError: baseline() missing 1 required positional argument: 'prediction'

In [ ]:
rand_frst_X.fit(X_train,Y_train)
prediction = rand_frst_X.predict(X_train)
print(accuracy_score(Y_train, prediction))
print(confusion_matrix(Y_train, prediction))

prediction = rand_frst_X.predict(X_test)
print(accuracy_score(Y_test, prediction))
print(confusion_matrix(Y_test, prediction))

prediction = rand_frst_X.predict(X_val)
print(accuracy_score(Y_val, prediction))
print(confusion_matrix(Y_val, prediction))



all_ones = np.ones(len(Y_test))
all_neg_ones = np.ones(len(Y_test)) * -1

print(f"Actual number of ones : {np.count_nonzero(Y_test == 1)}")
print(f"Predicted number of ones : {np.count_nonzero(prediction == 1)}")
print(f"Baseline if all were ones : {np.count_nonzero((all_ones==Y_test)) / len(Y_test)}")

print(f"Actual number of -1 : {np.count_nonzero(Y_test == -1)}")
print(f"Predicted number of -1 : {np.count_nonzero(prediction == -1)}")
print(f"Baseline if all were -1 : {np.count_nonzero((all_neg_ones==Y_test)) / len(Y_test)}")   

# prediction2 = rand_frst_class.predict(S_test)

# print(accuracy_score(Y_test, prediction2))
# print(confusion_matrix(Y_test, prediction2))


# rand_frst_class.fit(C_train,Y_train)    

# prediction3 = rand_frst_class.predict(C_test)

# print(accuracy_score(Y_test, prediction3))
# print(confusion_matrix(Y_test, prediction3))


0.998062015503876
[[240   0]
 [  1 275]]
0.5813953488372093
[[25 49]
 [23 75]]


ValueError: X has 3080 features, but DecisionTreeClassifier is expecting 280 features as input.

Hyperparameter tuning using RandomizedSearchCV

### Model Evaluation : Accuracy 

In [ ]:
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, mean_squared_error



rand_frst_class.fit(X_train,Y_train)    
prediction = rand_frst_class.predict(X_test)

print(accuracy_score(Y_test, prediction))
print(confusion_matrix(Y_test, prediction))



# rand_frst_class.fit(S_train,Y_train)    

# prediction2 = rand_frst_class.predict(S_test)

# print(accuracy_score(Y_test, prediction2))
# print(confusion_matrix(Y_test, prediction2))


# rand_frst_class.fit(C_train,Y_train)    

# prediction3 = rand_frst_class.predict(C_test)

# print(accuracy_score(Y_test, prediction3))
# print(confusion_matrix(Y_test, prediction3))




# np.set_printoptions(precision=2, suppress=True)




# print(predictions)

# print(np.array(Y_test['Log Returns']))
# errors  = abs(predictions - Y_test['Log Returns'])

# print('Average absolute error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
# mape = 100 * (errors / Y_test)
# Calculate and display accuracy
# accuracy = 100 - np.mean(mape)
# print('Accuracy:', round(accuracy, 2), '%.')



# Plot of the predicted log_returns vs actual returns
# plt.plot(Y_test['Log Returns'])
# plt.plot(predictions, 'o')
# plt.show()


# prediction = rand_frst_class.predict(X_test)
# mse = mean_squared_error(Y_test, rand_frst_class.predict(X_test))






0.5988372093023255
[[32 37]
 [32 71]]
